In [7]:
import requests
import pandas as pd
import os
import datetime
import openpyxl

def read_period_worksheet():
    period_worksheet = pd.read_excel("Periodo.xlsx")
    start_date = period_worksheet.iloc[0, 0].strftime("%Y-%m-%d")
    end_date = period_worksheet.iloc[0, 1].strftime("%Y-%m-%d")
    return start_date, end_date

start_date, end_date = read_period_worksheet()
extraction_period = start_date + "-" + end_date


url_base = f"https://api.movidesk.com/public/v1/tickets?token=a8d83bcc-5d05-44ed-be76-cd872d880dd2&$select=id,createdDate,clients,owner,chatWaitingTime,chatTalkTime,origin,status,protocol&$expand=clients($select=businessName,email),owner($select=businessName)&$filter=createdDate ge {start_date}T00:00:00.00Z and createdDate le {end_date}T23:59:59.00Z"

tickets = []
page = 0

while True:
    url = url_base + f"&$skip={page * 1000}"
    response = requests.get(url)

    if response.status_code == 200:
        page_tickets = response.json()
        if not page_tickets:
            break
        tickets.extend(page_tickets)
        page += 1
        print(f"Extraindo página {page} de tickets...")
    else:
        log_message = f"{datetime.datetime.now()}: Erro ao recuperar os dados: {response.status_code}\n"
        save_error_log(log_message)
        print(f"Erro ao recuperar os dados: {response.status_code}")
        break

df = pd.DataFrame(tickets)

# Dividir a coluna "clients" em "Fornecedor" e "E-mail Fornecedor"
df["Fornecedor"] = df["clients"].apply(lambda x: x[0]["businessName"] if x else None)
df["E-mail Fornecedor"] = df["clients"].apply(lambda x: x[0]["email"] if x else None)
df["Operador"] = df["owner"].apply(lambda x: x["businessName"] if x else None)

# Excluir colunas desnecessárias
df.drop(["clients", "owner"], axis=1, inplace=True)

# Renomear e reorganizar colunas
df.rename(columns={"id": "Ticket",
                   "protocol": "Procolo",
                   "createdDate": "Data de Abertura",
                   "chatWaitingTime": "Tempo de Espera",
                   "chatTalkTime": "Tempo de Atendimento",
                   "origin": "Origem",
                   "status": "Status"
                  }, inplace=True)

# Converter o formato da data na coluna "Data de Abertura"
df["Data de Abertura"] = pd.to_datetime(df["Data de Abertura"]).dt.strftime("%Y-%m-%d %H:%M:%S")

# Filtrar origem da interação
df = df[(df['Origem'] == 5) & (df['Status'] != 'Cancelado') & (df['Operador'].notna())]

# Reorganizar as colunas na ordem especificada
column_order = ["Ticket", "Operador", "Fornecedor", "E-mail Fornecedor", "Tempo de Espera", "Tempo de Atendimento", "Data de Abertura", "Origem", "Status", "Procolo"]
df = df[column_order]

# Excluir colunas desnecessárias
df.drop(["Origem", "Status", "Procolo"], axis=1, inplace=True)

# Criar resumo da qualidade dos dados
#quality_summary = pd.DataFrame(df.isnull().sum(), columns=["Valores em branco"])
#quality_summary["Porcentagem de valores em branco"] = (quality_summary["Valores em branco"] / len(df)) * 100

# Definir o nome base do arquivo
base_filename = f"Dados_Movidesk_{extraction_period}"

# Verificar se o arquivo já existe
if os.path.exists(base_filename + ".xlsx"):
    # Se o arquivo existir, adicionar um carimbo de data/hora ao nome do arquivo
    timestamp = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
    filename = base_filename + timestamp + ".xlsx"
else:
    # Se o arquivo não existir, usar o nome base do arquivo
    filename = base_filename + ".xlsx"

# Exportar os dados e o resumo da qualidade dos dados para a mesma planilha
with pd.ExcelWriter(filename) as writer:
    df.to_excel(writer, sheet_name="Movidesk Chat", index=False)
    #quality_summary.to_excel(writer, sheet_name="Resumo", index=True)
    



Extraindo página 1 de tickets...
Extraindo página 2 de tickets...
Extraindo página 3 de tickets...
Extraindo página 4 de tickets...
Extraindo página 5 de tickets...
Extraindo página 6 de tickets...
Extraindo página 7 de tickets...
Extraindo página 8 de tickets...
Extraindo página 9 de tickets...
Extraindo página 10 de tickets...
Extraindo página 11 de tickets...
Extraindo página 12 de tickets...
Extraindo página 13 de tickets...
Extraindo página 14 de tickets...
Extraindo página 15 de tickets...
Extraindo página 16 de tickets...
Extraindo página 17 de tickets...
Extraindo página 18 de tickets...
Extraindo página 19 de tickets...
Extraindo página 20 de tickets...
Extraindo página 21 de tickets...
Extraindo página 22 de tickets...
Extraindo página 23 de tickets...
Extraindo página 24 de tickets...
Extraindo página 25 de tickets...
Extraindo página 26 de tickets...
Extraindo página 27 de tickets...
Extraindo página 28 de tickets...


In [1]:
import pandas as pd

file_path = r"C:\Users\fcsantos\OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA\_Inteligência_Empresarial\Atividades_Felipe\API_Movidesk\Backup\04-2023 (1).xls"

df = pd.read_excel(file_path, nrows=10)

print(df.dtypes)

Ticket                   int64
Operador                object
Fornecedor              object
E-mail Fornecedor       object
Tempo de Espera          int64
Tempo de Atendimento     int64
Data de Abertura        object
dtype: object


In [4]:
import pandas as pd

file_path = r"C:\Users\fcsantos\INTELIGENCIA_EMPRESARIAL\MOVIDESK\Dados_Movidesk_2023-04-01-2023-04-30_20230518_122335.xlsx"

df = pd.read_excel(file_path, nrows=10)

print(df.dtypes)


Ticket                   int64
Operador                object
Fornecedor              object
E-mail Fornecedor       object
Tempo de Espera          int64
Tempo de Atendimento     int64
Data de Abertura        object
Origem                   int64
Status                  object
Procolo                  int64
dtype: object
